Three functions here:
oneHot ---> takes columns with CSV data and moves them to a OHE dataframe
plotableOHE ---> takes a column with CSV data and a "measure" column of numeric data and gives count and average of that measure for each category in the column
multiplePlotOHE ---> same as plotableOHE but can take and return multople "measure" columns

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import altair as alt

In [3]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [4]:
data = pd.read_csv("InitialClean.csv")

C:\Users\nfair\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8,18,19,20,21,22,23,57,58,59,60,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#INPUT: string name of column
#OUTPUT: dataframe with every unique entry as a one-hot encoding
#Notes: entries must be seperated by ";"
#       output will be in same order as dataframe passed in
#            and can be appended if desired
#       null entries will be categorized with a "1" in the "nan" column
#CHANGE 'DATA' TO WHATEVER THE MAIN DATAFRAME IS NAMED
def oneHot(columnName):
    tempData = pd.DataFrame(data[columnName].str.split(";"))
    tempData = tempData[columnName].tolist()
    for i in range(len(tempData)):
        if type(tempData[i])==float:
            tempData[i]=[str(tempData[i])]
    mlb = MultiLabelBinarizer()
    mlb.fit(tempData)
    oneHot = pd.DataFrame(mlb.transform(tempData), columns = mlb.classes_)
    return oneHot

In [6]:
currentLanguages = oneHot("LanguageWorkedWith")

In [7]:
#now you can do analysis. For example: 
currentLanguages.Python.mean()

0.30710636791259927

In [8]:
currentLanguages.describe()

,Assembly,Bash/Shell,C,C#,C++,CSS,Clojure,Cobol,CoffeeScript,Delphi/Object Pascal,...,Ruby,Rust,SQL,Scala,Swift,TypeScript,VB.NET,VBA,Visual Basic 6,nan
count,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.00000,98855.000000,98855.000000,98855.000000,...,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000
mean,0.058267,0.315331,0.182510,0.272662,0.201022,0.515695,0.01044,0.005968,0.026331,0.020485,...,0.080026,0.018785,0.451874,0.034596,0.063831,0.137838,0.053149,0.038541,0.031228,0.207587
std,0.234249,0.464650,0.386266,0.445331,0.400766,0.499756,0.10164,0.077025,0.160120,0.141651,...,0.271335,0.135766,0.497681,0.182755,0.244453,0.344732,0.224331,0.192500,0.173933,0.405581
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
#CHANGE 'DATA' TO WHATEVER THE MAIN DATAFRAME IS NAMED IN YOUR WORKSHEET
#for this function aggName and oheColumn are both single strings. 
#support for multiple aggregation columns is below
def plotableOHE(aggName, oheColumn):
    #use oneHot to oneHot the column of interest
    oheFrame = oneHot(oheColumn)
    #add the aggregate column (ie salary). 
    oheFrame[aggName] = data[aggName]
    #remove rows where the aggregate value is null
    oheFrame = oheFrame[oheFrame[aggName].notnull()]
    #get the counts of each label
    ones = np.ones(len(oheFrame[aggName]))
    countColumn = np.dot(ones,oheFrame)
    #get the total of each label
    outputFrame = pd.DataFrame(oheFrame[aggName].dot(oheFrame))
    outputFrame.insert(loc=0,column=oheColumn, value=outputFrame.index)
    outputFrame.reset_index(drop=True, inplace=True)
    countsName = oheColumn + "Counts"
    outputFrame.insert(loc=1,column=countsName, value=countColumn)
    #the aggName row will have squared in the dot product- we just drop it
    outputFrame=outputFrame[outputFrame[oheColumn]!=aggName]
    averageName = aggName+"Average"
    outputFrame[averageName] = outputFrame[aggName]/outputFrame[countsName]
    outputFrame.rename(columns = {aggName:aggName+"Sum"}, inplace = True)
    return outputFrame

In [11]:
#example
plotLangSalary = plotableOHE("ConvertedSalary", "LanguageWorkedWith")
plotLangSalary.head()

,LanguageWorkedWith,LanguageWorkedWithCounts,ConvertedSalarySum,ConvertedSalaryAverage
0,Assembly,2748.0,2.375183e+08,86433.161572
1,Bash/Shell,20408.0,2.190613e+09,107340.899941
2,C,9151.0,8.016037e+08,87597.391979
3,C#,16764.0,1.662746e+09,99185.494393
4,C++,10566.0,9.549368e+08,90378.271531


In [12]:
#aggNames is a list of numerical column inputs. Can only work with one OHE column
def multiplePlotOHE(aggNames, oheColumn):
    outputFrame = plotableOHE(aggNames[0], oheColumn)
    for agg in aggNames[1:]:
        newOutputFrame = plotableOHE(agg, oheColumn)
        outputFrame = outputFrame.join(newOutputFrame.set_index(oheColumn), on=oheColumn, rsuffix='_other')
        dropColumn = oheColumn + "Counts_other"
        outputFrame.drop(dropColumn, axis=1, inplace=True)
    return outputFrame
        

In [13]:
#example
plotTwo = multiplePlotOHE(["ConvertedSalary","CompanySizeQuantSmall"], "LanguageWorkedWith")
plotTwo.head(20)

,LanguageWorkedWith,LanguageWorkedWithCounts,ConvertedSalarySum,ConvertedSalaryAverage,CompanySizeQuantSmallSum,CompanySizeQuantSmallAverage
0,Assembly,2748.0,2.375183e+08,86433.161572,7179467.0,2002.082264
1,Bash/Shell,20408.0,2.190613e+09,107340.899941,48576478.0,1973.931407
2,C,9151.0,8.016037e+08,87597.391979,24741906.0,2026.032263
3,C#,16764.0,1.662746e+09,99185.494393,36856015.0,1709.700561
4,C++,10566.0,9.549368e+08,90378.271531,28054909.0,2034.438651
5,CSS,30864.0,2.950737e+09,95604.488628,61080078.0,1569.334755
6,Clojure,696.0,8.755012e+07,125790.397989,1465526.0,1829.620474
7,Cobol,302.0,3.815877e+07,126353.529801,1107820.0,2643.961814
8,CoffeeScript,1752.0,2.004303e+08,114400.859589,2412961.0,1186.313176
9,Delphi/Object Pascal,1071.0,9.818206e+07,91673.257703,1659126.0,1151.371270


In [17]:
example_chart = alt.Chart(plotLangSalary, title='Salary By Programming Language').mark_bar(opacity=0.7, tooltip=True).encode(
    x='LanguageWorkedWith',
    y=alt.Y('ConvertedSalaryAverage:Q', stack=None, title='Average Salary ($)'),
    #color='variable'
)

In [18]:
example_chart

alt.Chart(...)

In [19]:
#anotherExample:
#example
plotLangSalary = plotableOHE("ConvertedSalary", "UndergradMajor")
plotLangSalary.head()

,UndergradMajor,UndergradMajorCounts,ConvertedSalarySum,ConvertedSalaryAverage
0,Business,1044.0,1.017447e+08,97456.597701
1,Computer Science,26788.0,2.561723e+09,95629.497910
2,Engineering (non-computer),3576.0,3.290210e+08,92008.123322
3,Fine Arts,726.0,9.182990e+07,126487.458678
4,Health Science,129.0,1.107930e+07,85886.077519


In [21]:
example_chart2 = alt.Chart(plotLangSalary, title='Salary By Programming Language').mark_bar(opacity=0.7, tooltip=True).encode(
    x='UndergradMajor',
    y=alt.Y('ConvertedSalaryAverage:Q', stack=None, title='Average Salary ($)'),
    #color='variable'
)
example_chart2

alt.Chart(...)

In [ ]:
#SEE HOW FAST THAT WAS!!!???